In [161]:
import os
import spacy
import nltk
import nltk.tokenize
import re
import unidecode
import collections
import numpy as np
import csv
import matplotlib.pyplot as plt


In [202]:
n = 317 # Choose a number for a song to inspect.

In [163]:
def read_data(csvfile):
	# Result: list (= row) of lists, without header.
	data = []
	headers = True
	with open(csvfile) as stream:
		reader = csv.reader(stream, delimiter=',')
		for row in reader:
			if not headers:
				data.append(row)

			headers = False

	return data

In [192]:
liedjesData = np.array(read_data('../../data/liedjes/liedjes.csv')) # create np.array to be able to order lexicographically
liedjesData = liedjesData[liedjesData[:, 1].argsort()] # order lexicographically on identifier just like the textlist
print(liedjesData[n])

['https://zotero.org/groups/2707622/items/SGA2CMRR'
 'utrecht1851-feestzangen1854-15' 'VRIENDSCHAP' 'Io Vivat' '1854'
 'B. Lageman' 'utrecht1851']


In [165]:
def getlistOfTexts(rootdir):
    # input: directory with (subdirectory with) TXT-files to be handled
	# output: list of files lexicographically ordered on path-name

    files_all = []
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not file.endswith('.txt'):
                continue
            fn = os.path.join(subdir, file)
            files_all.append(fn)

    files_all = sorted(files_all)
    return files_all

liedjesList = getlistOfTexts('../../data/liedjes')

In [198]:
print(liedjesList[n])

../../data/liedjes/utrecht1851-feestzangen1854/utrecht1851-feestzangen1854-14.txt


In [167]:
def texts2list(listOfTexts):
	# input: list of files lexicographically ordered on path-name
    # output: list of texts

	texts = []
	for file in listOfTexts:
		with open(file) as stream:
			text = stream.read()
		texts.append(text)

	return texts

liedjes = texts2list(liedjesList)

In [203]:
print(liedjes[n])

Huldezang

O, achtbaar Hoofd van deze stad !
Beschermheer van deez’ Kring !
Gun aan deez’ schaar vam vrienden, dat
Zij U met lof bezing’:
O, God! zie met een Vaderoog
Op den Beschermer neêr;
Ons hart klimt biddend naar omhoog :
Verhoor die bede, Heer!

Komt, feestgenooten! zingen wij
Ter eer van elks Patroon ! 
Verheft uw lied, verheugd en blij:
De dankbaarheid is schoon !
Wij danken U, o edel tal!
Voor 't goede aan ons verrigt;
Ons hart blijv’, wat ook sta of vall’,
Geheel aan U verpligt.

Door dankbaarheid geheel ontgloeid,
Verheft ons hart den zang
Ter eer van U, die, onvermoeid
Behartigd’ ons belang.
Gods zegen blijv’ gestaâg U bij,
O, VENMAN! op Uw baan!
Neemt van deez’ blijde vriendenrei
Den gullen danktoon aan.

Voorstanders! blijft voor onzen kring,
(Ook dit nog bidden wij,)
De schoonste en rijkste zegening,
Dan maakt Uw steun ons blij.
Van onspoed vrij en ’s werelds zuur
Snell’ steeds Uw leven voort;
Uw’ harten word’ door ’t edel vuur
Van menschenmin bekoord.



In [169]:
def lemmatize(texts):   
    # input: list of texts
    # output: list of lemmatized texts. Lemmatization by spaCy.

    nlp = spacy.load("nl_core_news_sm") # create spaCy processor named 'nlp' based on small model for Dutch
    nlp.Defaults.stop_words |= {"deez", "zoo", "hoezee"} # optionally: add stopwords to processor

    result = []                                                         
    for text in texts:                                                     

        nlp.max_length = len(text)  
        doc = nlp(text) # tokenize and annotate 'text' with processor named 'nlp'

        new_text = []
        for token in doc: 
            if token.is_alpha: # keep tokens with alphanumerical characters (so no numbers or punctuation)
                if not token.is_stop: # remove stopwords
                    new_text.append(token.lemma_) # get the word in the lemma and add it to the list of words

        final = " ".join(new_text) # transform list of words into a string concatenating all listitems
        result.append(final) # add string to the list of lemmatized texts

    return result


In [170]:
lemmatized_liedjes = lemmatize(liedjes)

In [195]:
print(lemmatized_liedjes[n])

nut genoegen derde jaar weêr vrolijk elkaêr kopperfeest goddank verbond stand houden dee staan nut bevatten nut genieten dees vriendenrij lijden komen ziekengeld regten prijs stellen gevoel heerlijk doel laten zóó voorwaarts gaan inslaan baan ieders nut genoegen kosterschaar klinken feestbokaal zaal genoegen zang muzijk wijn kopperfeest komen broeder glas vatten gevuld druivennat drinken zaâm koster naam welaan welaan welaan moog gelukkig gaan kring o god bidden behoed dee vriendenrij jaar jaar dank elkaêr


In [172]:
def is_punctuation(string):
	# Result: A boolean for being a punctuation mark or not
	return re.compile(r'[^\w\s]+$').match(string) is not None

def remove_diacritics(string):
	# Result: A string without diacritics (accents)
	return unidecode.unidecode(string)

def preprocess_text(string):
	# Result: list of tokens in a string (= "tokenized corpus")
	string = string.lower()
	tokens = nltk.tokenize.word_tokenize(string, language="dutch")
	tokens = [token for token in tokens if not is_punctuation(token)]
	tokens = [remove_diacritics(token) for token in tokens]

	return tokens

tokenized_liedjes = [preprocess_text(liedje) for liedje in lemmatized_liedjes]

In [196]:
print(tokenized_liedjes[n])

['nut', 'genoegen', 'derde', 'jaar', 'weer', 'vrolijk', 'elkaer', 'kopperfeest', 'goddank', 'verbond', 'stand', 'houden', 'dee', 'staan', 'nut', 'bevatten', 'nut', 'genieten', 'dees', 'vriendenrij', 'lijden', 'komen', 'ziekengeld', 'regten', 'prijs', 'stellen', 'gevoel', 'heerlijk', 'doel', 'laten', 'zoo', 'voorwaarts', 'gaan', 'inslaan', 'baan', 'ieders', 'nut', 'genoegen', 'kosterschaar', 'klinken', 'feestbokaal', 'zaal', 'genoegen', 'zang', 'muzijk', 'wijn', 'kopperfeest', 'komen', 'broeder', 'glas', 'vatten', 'gevuld', 'druivennat', 'drinken', 'zaam', 'coster', 'naam', 'welaan', 'welaan', 'welaan', 'moog', 'gelukkig', 'gaan', 'kring', 'o', 'god', 'bidden', 'behoed', 'dee', 'vriendenrij', 'jaar', 'jaar', 'dank', 'elkaer']


In [174]:
def replace(listOfWords, replaceWords):
    i = 0
    for word in listOfWords:
        if word in replaceWords:
            listOfWords[i] = replaceWords[word]
        i = i + 1

    return listOfWords

def remove(listOfWords, removeWords):
    i = 0
    for word in listOfWords:
        if word in removeWords:
            listOfWords.pop(i)
        i = i + 1

    return listOfWords

In [197]:
replaceDict = {"koster":"coster", "costers":"coster", "kosters":"coster", "vreugd":"vreugde"}
removeList  = []

processed_liedjes = []
for liedje in tokenized_liedjes:
    processed_liedje1 = replace(liedje, replaceDict)
    processed_liedje2 = remove(processed_liedje1, removeList)
    processed_liedjes.append(processed_liedje2)

print(processed_liedjes[n])

['nut', 'genoegen', 'derde', 'jaar', 'weer', 'vrolijk', 'elkaer', 'kopperfeest', 'goddank', 'verbond', 'stand', 'houden', 'dee', 'staan', 'nut', 'bevatten', 'nut', 'genieten', 'dees', 'vriendenrij', 'lijden', 'komen', 'ziekengeld', 'regten', 'prijs', 'stellen', 'gevoel', 'heerlijk', 'doel', 'laten', 'zoo', 'voorwaarts', 'gaan', 'inslaan', 'baan', 'ieders', 'nut', 'genoegen', 'kosterschaar', 'klinken', 'feestbokaal', 'zaal', 'genoegen', 'zang', 'muzijk', 'wijn', 'kopperfeest', 'komen', 'broeder', 'glas', 'vatten', 'gevuld', 'druivennat', 'drinken', 'zaam', 'coster', 'naam', 'welaan', 'welaan', 'welaan', 'moog', 'gelukkig', 'gaan', 'kring', 'o', 'god', 'bidden', 'behoed', 'dee', 'vriendenrij', 'jaar', 'jaar', 'dank', 'elkaer']


In [176]:
def extract_vocabulary(tokenized_corpus, min_count=1, max_count=float('inf')):
	# Result: list of unique words derived from a list of lists of strings
	vocabulary = collections.Counter()
	for document in tokenized_corpus:
		vocabulary.update(document)
	vocabulary = { 
		word for word, count in vocabulary.items()
		if count >= min_count and count <= max_count
	}

	return sorted(vocabulary)

liedjesVocabulary = extract_vocabulary(processed_liedjes, min_count=1)
# liedjesVocabulary

In [177]:
topicList = [['jenever',  'alcohol',  'drank',  'spraakwater',  'aperitief',  'biertje',  'bier',  'brouwsel',  'gerstenat',  'donker_bier',  'stout',  'stout',  'vatbier',  'pils',  'pilsener',  'bokbier',  'pils',  'pilsener',  'mede',  'appelwijn',  'cider',  'eigenstook',  'koemis',  'likeur',  'absint',  'amaretto',  'benedictijner',  'chartreuse',  'Drambuie',  'drank',  'arak',  'aqua_vitae',  'brandewijn',  'brandy',  'appeldrank',  'appelbrandewijn',  'Calvados',  'cognac',  'grappa',  'vuur_water',  'wie_verre_reizen_doet,_kan_veel_verhalen',  'gin',  'ouzo',  'rum',  'grog',  'toddy',  'Schnaps',  'tequila',  'vodka',  'wodka',  'whisky',  'Drambuie',  'cocktail',  'bloody_mary',  'Martini',  'martini',  'gin-tonic',  'grog',  'toddy',  'punch',  'Advocaat',  'advokaat',  'bocht',  '-halve',  'druivenat',  'druivennat',  'wijn',  'roos',  'Bourgondië',  'dessertwijn',  'Marsala',  'porto',  'sherry',  'xeres',  'xereswijn',  'gekruide_wijn',  'bocht',  'rode_wijn',  'schuimwijn',  'champagne',  'rassen',  'vermout',  'wijnjaar',  'wijnoogst',  'witte_wijn',  'Moezel',  'Moselle',  'rijnwijn',  'riesling',  'zak',  'Sauterne',  'drinkglas',  'glaasje',  'glas',  'bierglas',  'likeurglaasje',  'roemer',  'romer',  'wijnbeker',  'wijnroemer',  'borrelglas',  'jeneverglas',  'borrel',  'bel',  'fluit',  'beker',  'bak',  'kop',  'tas',  'beker',  'kelk',  'miskelk',  'koffiekop',  'koffiekopje',  'mokkakopje',  'espressokopje',  'kylix',  'theekopje',  'drinken',  'oplebberen',  'oplikken',  'opslobberen',  'opslorpen',  'opslurpen',  'aanzuigen',  'opzuigen',  'zuigen',  'aanstoten',  'klinken',  'proosten',  'toasten',  'toosten'], ['vriendenkring',  'samenleving',  'gilde',  'gezelschap',  'sociëteit',  'orde',  'club',  'vereniging',  'boekenclub',  'vrouwenclub',  'zusterschap',  'zeilvereniging',  'partner',  'lid',  'compagnon',  'medewerker',  'bepaling',  'aannemen',  'lid_maken_van',  'affiliëren',  'bondgenoot',  'bondgenote',  'vrind',  'compagnon',  'zakenvriend',  'vriend',  'vennoot',  'deelgenoot',  'compagnon',  'partner',  'mede-werker',  'gildebroeder',  'gildenbroeder',  'medewerker',  'vakbroeder',  'vakgenoot',  'collega',  'ambtgenoot',  'ambtsbroeder',  'ambtsgenoot',  'confrater',  'confrère',  'gildebroeder',  'gildenbroeder',  'medebroeder',  'vakbroeder',  'vakgenoot',  'lid',  'directielid',  'broer',  'broeder',  'vrijmetselaar',  'gevolmachtigde',  'commissaris',  'bestuurslid',  'commissielid',  'gemeenteraadslid',  'raadslid',  'stamgenoot',  'deelhebber',  'deelnemer',  'participant',  'genodigde',  'aanwezige',  'feestganger',  'feestvierder',  'bruidegom',  'deelgenoot',  'Pooler',  'makker',  'medespeler',  'ploegmaat',  'ploegmakker',  'teamgenoot',  'broederlijkheid',  'broederschap',  'gemeenschap',  'saamhorigheid',  'verbondenheid',  'kameraadschap',  'vriendschap',  'vriendschappelijkheid',  'vriendschapsband',  'vriendschapsbanden',  'kameraadschap'], ['drukkunst',  'boekdrukkunst',  'drukken',  'lithograferen',  'steendrukken',  'stencilen',  'drukker',  'typograaf',  'letterzetter',  'machinezetter',  'typograaf',  'zetter',  'binder',  'boekbinder',  'drukpers',  'pers',  'boekdrukkerij',  'drukkerij',  'drukpers'], ['beeld',  'beeldhouwwerk',  'sculptuur',  'borstbeeld',  'buste',  'camee',  'schriftteken',  'glief',  'mobiel',  'mobile',  'afgietsel',  'gietsel',  'replica',  'reliëf',  'standbeeld',  'beeld',  'Sfinx',  'sfinx',  'metaal',  'alkalimetaal',  'aluminium',  'americium',  'antimonium',  'antimoon',  'barium',  'bariumoxyde',  'berkelium',  'beryllium',  'astin',  'bismut',  'cadmium',  'calcium',  'californium',  'cerium',  'caesium',  'cesium',  'chromium',  'chroom',  'chromium',  'chroom',  'kobalt',  'koperbruin',  'koper',  'cirium',  'curium',  'dysprosium',  'einsteinium',  'erbium',  'europium',  'fermium',  'francium',  'gadolinium',  'gallium',  'hafnium',  'metal',  'zware_metalen',  'holmium',  'indium',  'iridium',  'strijkijzer',  'ijzer',  'goteling',  'gietijzer',  'smeedijzer',  'kunstsmeedwerk',  'lanthanium',  'lanthaan',  'ik_zoek_een_baan',  'lood',  'lithium',  'lutetium',  'magnesium',  'mangaan',  'kwik',  'kwikzilver',  'mercurius',  'molybdeen',  'neodymium',  'neodynium',  'neptunium',  'nikkel',  'niobium',  'niobium',  'goud',  'platina',  'witgoud',  'een_G',  'zilver',  'osmium',  'palladium',  'polonium',  'kalium',  'praseodymium',  'promethium',  'protactinium',  'radium',  'renium',  'rhenium',  'rhodium',  'rodium',  'rubidium',  'rutenium',  'ruthenium',  'samarium',  'scandium',  'natrium',  'strontium',  'tantaal',  'tantalium',  'technetium',  'terbium',  'tallium',  'thallium',  'thorium',  'thulium',  'blik',  'tin',  'titaan',  'titanium',  'wolfraam',  'wolframiet',  'uraan',  'uranium',  'vanadium',  'ytterbium',  'yttrium',  'zink',  'zirkonium',  'zirkoon',  'steen',  'deksteen',  'hoeksteen',  'zerk',  'grafsteen',  'maalsteen',  'molensteen',  'monoliet',  'plavuis',  'steen',  'stoeptegel',  'tegel',  'trottoirtegel',  'inst',  'kasseisteen',  'straatsteen',  'stoeprand',  'hardsteen',  'plavuis',  'steen',  'tegel',  'tichel',  'vloertegel',  'slijpsteen',  'wetsteen',  'slijpsteen',  'oliesteen'], ['licht',  'verlichting',  'wetenschap',  'duisternis',  'lichtstraal',  'wijsheid',  'vrijdom',  'vrijheid',  'mondigheid',  'onafhankelijkheid',  'gesloten_staatshuishouding',  'autarkie',  'zelfbeschikkingsrecht',  'zelfbestuur',  'zelfbeschikking',  'self-government',  'selfgovernment',  'zelfvertrouwen',  'zelfvoorziening',  'autarkie',  'vergunning',  'bandeloosheid',  'fakkel',  'flambouw',  'toorts',  'stralen',  'schijnen',  'branden',  'glimmen'], ['uitvinden',  'uitvinding',  'vinding',  'vondst',  'uitdenken',  'vinding',  'doorbraak',  'ontdekking',  'vondst'], ['Nederland',  'Holland',  'Nederlandsch',  'Hollandsch',  'Nederlands',  'Hollands',  'Neerlands',  '[[land]]_[[van]]_[[oorsprong]]',  'land_van_herkomst',  'geboortegrond',  'geboorteland',  'moederland',  'patria',  'thuisland',  'vaderland'], ['eerelid',  'eereleden',  'beschermheer',  'donateur',  'abonnee',  'geabonneerde',  'ondertekenaar',  'huurder',  'baas',  'patroon']]

topicsVocabulary = []
for topic in topicList:
    topicsVocabulary = topicsVocabulary + topic

preciseLiedjesVocabulary = list(set(liedjesVocabulary) & set(topicsVocabulary))
# preciseLiedjesVocabulary

In [178]:
def corpus2dtm(tokenized_corpus, vocabulary):
	# Result: Document Term Matrix: 
	# rows being documents in tokenized_corpus, columns being the words in vocabulary, 
	# values being the count of the word in the document
	document_term_matrix = []
	for document in tokenized_corpus:
		document_counts = collections.Counter(document)
		row = [document_counts[word] for word in vocabulary]
		document_term_matrix.append(row)

	return document_term_matrix

dtm_liedjes = np.array(corpus2dtm(processed_liedjes, preciseLiedjesVocabulary))
# dtm_liedjes

In [179]:
# print vector of song $n
for i in range(len(dtm_liedjes[n])):
    print(preciseLiedjesVocabulary[i], dtm_liedjes[n][i])

vinding 0
broeder 0
orde 0
baas 0
makker 0
kop 0
wetenschap 0
drukkunst 0
vriendschap 0
wijn 0
glas 0
patroon 0
pers 0
drukker 0
vriendschapsband 0
onafhankelijkheid 0
bier 0
toasten 0
eereleden 0
schijnen 0
beschermheer 0
uitdenken 0
steen 0
glimmen 0
stralen 0
wijsheid 0
beker 0
zilver 0
verlichting 0
vrind 0
roos 0
zuigen 0
kelk 0
beeld 0
donateur 0
uitvinden 0
fakkel 0
boekdrukkunst 0
vriend 0
branden 0
stout 0
typograaf 0
goud 0
vaderland 1
lid 0
roemer 0
binder 0
zak 0
zink 0
gezelschap 0
druivennat 0
palladium 0
klinken 0
drinken 0
vrijheid 0
broederschap 0
letterzetter 0
blik 0
standbeeld 0
metaal 0
drank 0
drukkerij 0
lichtstraal 0
drukpers 0
vriendenkring 0
zetter 0
geboortegrond 0
uitvinding 0
duisternis 0
licht 0
toorts 0
drukken 0


In [180]:
def topics2vectors(topicList, vocabulary):
    result = []
    for topic in topicList:
        topicVector = []
        for term in vocabulary:
            if term in topic:
                topicVector.append(1)
            else:
                topicVector.append(0)
        result.append(topicVector)

    return result

topicVectors = np.array(topics2vectors(topicList, preciseLiedjesVocabulary))


In [181]:
document_topic_distance_matrix = []
all_values = []
for i in range(len(liedjesList)):
    a = dtm_liedjes[i]
    row = []
    for j in range(len(topicVectors)):
        b = topicVectors[j]
        if np.linalg.norm(a) > 0:
            cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
        else:
            cos_sim = 0.0
        row.append(cos_sim)
        if cos_sim < 1:
            all_values.append(cos_sim)
    document_topic_distance_matrix.append(row)



In [204]:
liedjesListTopic = []
for j in range(len(topicVectors)):
    l = []
    value = 0
    for i in range(len(liedjesList)):
        if value < document_topic_distance_matrix[i][j]: 
            value = document_topic_distance_matrix[i][j]
            maxdoc = i
    print('---------')
    print('topic: ', j)
    print('aantal terms vd topic :', len(topicList[j]))
    print('belangrijkste liedje  :', maxdoc)
    print('afstand topic-liedje  :', value)
    print('vector topic-liedje   :', document_topic_distance_matrix[maxdoc])



---------
topic:  0
aantal terms vd topic : 132
belangrijkste liedje  : 354
afstand topic-liedje  : 0.5103103630798288
vector topic-liedje   : [0.5103103630798288, 0.12309149097933275, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---------
topic:  1
aantal terms vd topic : 85
belangrijkste liedje  : 148
afstand topic-liedje  : 0.6030226891555273
vector topic-liedje   : [0.0, 0.6030226891555273, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---------
topic:  2
aantal terms vd topic : 19
belangrijkste liedje  : 318
afstand topic-liedje  : 0.5853694070049636
vector topic-liedje   : [0.1386750490563073, 0.08362420100070908, 0.5853694070049636, 0.0, 0.0, 0.1386750490563073, 0.0, 0.0]
---------
topic:  3
aantal terms vd topic : 156
belangrijkste liedje  : 49
afstand topic-liedje  : 0.629940788348712
vector topic-liedje   : [0.0, 0.0, 0.0, 0.629940788348712, 0.0, 0.0, 0.0, 0.0]
---------
topic:  4
aantal terms vd topic : 29
belangrijkste liedje  : 41
afstand topic-liedje  : 0.46291004988627577
vector topic-liedje   : [0.0,